<a href="https://colab.research.google.com/github/roshan-varadhan/spotify-playlist-recommender/blob/main/spotify_playlist_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Importing Libraries**

In [ ]:
!pip install spotipy
!pip install python-dotenv

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import os
from dotenv import load_dotenv
from pathlib import Path

# Connecting to Spotify

In [ ]:
#Authentication
load_dotenv(dotenv_path=Path('/content/variables.env'))
cid = os.getenv('CLIENT_ID')
secret = os.getenv('CLIENT_SECRET')

client_credentials_mgmt = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_mgmt)

## Playlist Dataframe

In [ ]:
playlist_link = input("Enter the playlist link: ")

# Extract playlist URI from the link
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

# Function to handle retries
def fetch_with_retry(func, *args, retries=5, delay=1, backoff=2, **kwargs):
    for i in range(retries):
        try:
            return func(*args, **kwargs)
        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 429:
                retry_after = int(e.headers.get('Retry-After', delay))
                print(f"Rate limited. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
                delay *= backoff
            else:
                raise e
    raise Exception("Max retries exceeded")

# Fetch playlist details
playlist_details = fetch_with_retry(sp.playlist, playlist_URI)

# Get playlist name and user display name
playlist_name = playlist_details['name']
user_display_name = playlist_details['owner']['display_name']

# Fetch playlist tracks
tracks = fetch_with_retry(sp.playlist_tracks, playlist_URI)["items"]

# Extract track details
track_uris = [item["track"]["uri"] for item in tracks]

# Batch URIs for audio features
def get_audio_features(track_uris):
    audio_features = []
    for i in range(0, len(track_uris), 100):
        batch = track_uris[i:i+100]
        features = fetch_with_retry(sp.audio_features, batch)
        audio_features.extend(features)
    return audio_features

# Fetch audio features for the tracks
audio_features = get_audio_features(track_uris)

# Combine track details with audio features
track_data = []
for item, features in zip(tracks, audio_features):
    if features:
      track = item["track"]
      track_name = track["name"]
      artist_name = track["artists"][0]["name"]  # Assuming the first artist is the main artist
      track_uri = track["uri"]

      # Fetch audio features for the track
      audio_features = sp.audio_features(track_uri)[0]
      danceability = audio_features['danceability']
      energy = audio_features['energy']
      liveliness = audio_features['liveness']
      key = audio_features['key']
      mode = audio_features['mode']
      loudness = audio_features['loudness']
      speechiness = audio_features['speechiness']
      acousticness = audio_features['acousticness']
      instrumentalness = audio_features['instrumentalness']
      valence = audio_features['valence']

      track_data.append([track_name, artist_name, danceability, energy, liveliness, key, mode, loudness, speechiness, acousticness, instrumentalness, valence])

# Create a DataFrame
df_playlist = pd.DataFrame(track_data, columns=["Track Name", "Artist", "Danceability", "Energy", "Liveliness","Key","Mode","Loudness","Speechiness","Acousticness","Instrumentalness","Valence"])
df_playlist.head()

## Loading the Data into a Pandas DataFrame

In [ ]:
df = pd.read_csv("playlist_2010to2023.csv", encoding='latin-1')
df.dropna(inplace=True)
# Drop Unnecessary Columns
df.drop(['year','artist_genres','playlist_url','track_id','track_popularity','artist_id','artist_popularity','time_signature', 'duration_ms'], axis=1, inplace=True)
df.head()


## Dropping Features and Combining Data

In [ ]:
# Create a new DataFrame with only necessary columns
dropped_df = df[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]


# Congregate playlist averages into one representative 'song' to be used for comparison
dropped_df_playlist = dropped_df.drop(['Track Name', 'Artist'])
means= dropped_df_playlist.mean()
dropped_df.loc[len(dropped_df.index)] = means
df.loc[len(df.index)] = [playlist_name,'', user_display_name, means['danceability'], means['energy'], means['key'], means['loudness'], means['mode'], means['speechiness'], means['acousticness'], means['instrumentalness'], means['liveness'], means['valence'], means['tempo']]

In [ ]:
# Normalizing numerical feature columns
numerical_columns = ['loudness', 'tempo','key']

# Create a copy of the dropped_df
final_df = dropped_df

# Apply min-max scaling to the selected numerical feature columns
final_df[numerical_columns] = (dropped_df[numerical_columns] - dropped_df[numerical_columns].min()) / (dropped_df[numerical_columns].max() - dropped_df[numerical_columns].min())
final_df

# Data Analyzation

## Clustering the Data

In [ ]:
# Function that works out optimum number of clusters
def optimise_k_means(data, max_k):
    means = []
    inertias = []

    for k in range(1, max_k+1):
        kmeans = KMeans(n_clusters=k, n_init=max_k)
        kmeans.fit(data)

        means.append(k)
        inertias.append(kmeans.inertia_)

    # Generate the elbow plot
    plt.figure(figsize=(10, 5))  # Create a new figure
    plt.plot(means, inertias, 'o-')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')
    plt.grid(True)
    plt.show()

In [ ]:
optimise_k_means(final_df, 10)

## Applying K-Means Clustering

In [ ]:
# Initialize a KMeans model with 4 clusters
kmeans = KMeans(n_clusters=10, n_init='auto')

# Fit the KMeans model to the data in 'final_df'
kmeans.fit(final_df)

# Assign cluster labels to each data point and add the 'Cluster' column to the original DataFrame
df['Cluster'] = kmeans.labels_
final_df['Cluster'] = kmeans.labels_

## Model Visualization

In [ ]:
# Perform PCA for dimensionality reduction
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(final_df.drop('Cluster', axis=1)) # Exclude the cluster labels

# Add the reduced components to the DataFrame
final_df['pca_1'] = reduced_features[:, 0]
final_df['pca_2'] = reduced_features[:, 1]

# Create a scatter plot
plt.scatter(final_df['pca_1'], final_df['pca_2'], c=final_df['Cluster'], cmap='viridis')
plt.title('KMeans Clustering Results using PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

## Song Recommendations

In [ ]:
def find_song_recommendation(track_name, df):
    try:
        # Attempt to find the index of the first occurence of 'track_name' in the 'Track' column of 'df'
        track_index = df[df['track_name'] == track_name].index[0]

        # Create a filter to select rows in 'df' that belong to the same cluster as the provided track
        filter = (df['Cluster'] == df.loc[track_index]['Cluster'])

        # Apply the filter to 'df' to get a DataFrame containing songs from the same cluster
        filtered_df = df[filter]

        # Generate song recommendations by randomly selecting tracks from the same cluster
        for i in range(50):
            # Randomly sample a track from the shuffled DataFrame
            recommendation = filtered_df.sample()
            # Print the recommended track's title and artist
            print("'" + recommendation.iloc[0]['track_name'] + "' by " + recommendation.iloc[0]['artist_name'])
    except:
      print("Track not found in the dataset.")

In [ ]:
find_song_recommendation(playlist_name, df)